In [3]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.pretrained import *

import warnings
warnings.filterwarnings("ignore")

from pyspark.context import SparkContext 
from pyspark import SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os 
os.environ['PYSPARK_PYTHON'] = "/root/miniconda3/envs/sp/bin/python"

spark = SparkSession.builder\
    .master("local[*]")\
    .config("spark.driver.memory", "16G")\
    .config("spark.driver.maxResultSize", "0")\
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
    .config("spark.kryoserializer.buffer.max", "2000m")\
    .config("spark.jsl.settings.pretrained.cache_folder", "sample_data/pretrained")\
    .config("spark.jsl.settings.storage.cluster_tmp_dir", "sample_dataorage")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.4.1")\
    .config(
    "spark.jars",
    "/data/jupyter-data/lab08/kafka-clients-3.4.0.jar,/data/jupyter-data/lab08/spark-sql-kafka-0-10_2.12-3.3.1.jar, \
    /data/jupyter-data/lab08/spark-token-provider-kafka-0-10_2.12-3.3.1.jar, \
    /data/jupyter-data/lab08/commons-pool2-2.11.1.jar") \
    .config("spark.executorEnv.PYSPARK_PYTHON","/root/miniconda3/3/envs/sp/bin/python") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .appName("Pro2").getOrCreate()

:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/Lenovo1/.ivy2/cache
The jars for the packages stored in: /Users/Lenovo1/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e4d94a14-6e46-432c-86b3-bcf9bb7c30c5;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.4.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.16.0 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#fa

24/07/26 11:04:09 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
import pyspark

In [4]:
MODEL_NAME = 'distilbert-base-cased-distilled-squad'

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context", "true_ans"]) \
    .setOutputCols(["document_question", "document_context", "document_ans"])

spanClassifier_loaded = DistilBertForQuestionAnswering.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

24/07/26 11:11:29 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/07/26 11:11:29 INFO SharedState: Warehouse path is 'file:/Users/Lenovo1/Desktop/25%20summer%20intern/QA-system/code/spark-warehouse'.
24/07/26 11:11:29 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 221.7 KiB, free 2.2 GiB)
24/07/26 11:11:30 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 32.7 KiB, free 2.2 GiB)
24/07/26 11:11:30 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on xiongjindiannao.lan:61350 (size: 32.7 KiB, free: 2.2 GiB)
24/07/26 11:11:30 INFO SparkContext: Created broadcast 0 from textFile at ReadWrite.scala:587


Py4JJavaError: An error occurred while calling o50.load.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/Lenovo1/Desktop/25 summer intern/QA-system/code/distilbert-base-cased-distilled-squad_spark_nlp/metadata
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at o

In [5]:
schema = StructType([
    StructField("question", StringType(), True),
    StructField("context", StringType(), True),
    StructField("true_ans", StringType(), True)
])

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "pro2_13") \
    .load()

json_stream = kafka_stream.select(from_json(kafka_stream.value.cast("string"), schema).alias("data"))
qa_stream = json_stream.select("data.question", "data.context", "data.true_ans")

query = qa_stream.writeStream \
    .outputMode("append") \
    .format("memory") \
    .option("queryName", "QA_model") \
    .trigger(processingTime = '5 seconds')
    
query.start()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

24/07/26 11:33:53 INFO BlockManagerInfo: Removed broadcast_0_piece0 on xiongjindiannao.lan:61350 in memory (size: 32.7 KiB, free: 2.2 GiB)
24/07/26 14:17:50 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 926620 ms exceeds timeout 120000 ms
24/07/26 14:17:50 WARN SparkContext: Killing executors is not supported by current scheduler.
24/07/26 14:17:57 INFO Executor: Told to re-register on heartbeat
24/07/26 14:17:57 INFO BlockManager: BlockManager BlockManagerId(driver, xiongjindiannao.lan, 61350, None) re-registering with master
24/07/26 14:17:57 INFO BlockManagerMaster: Registering BlockManager BlockManagerId(driver, xiongjindiannao.lan, 61350, None)
24/07/26 14:17:57 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.

In [4]:
df1 = spark.sql("SELECT * FROM QA_model")
df1.show(5, truncate=False)

+----------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------+
|question                                                        |context                                                                                         

In [9]:
result = pipeline.fit(df1).transform(df1)
df2 = result.select("question", "context", "true_ans", "answer.result")
df2.show(30)

+--------------------+--------------------+--------------------+--------------------+
|            question|             context|            true_ans|              result|
+--------------------+--------------------+--------------------+--------------------+
|In what country i...|The Normans (Norm...|              France|            [France]|
|When were the Nor...|The Normans (Norm...|10th and 11th cen...|[10th and 11th ce...|
|From which countr...|The Normans (Norm...|Denmark, Iceland ...|[Denmark , Icelan...|
|Who was the Norse...|The Normans (Norm...|               Rollo|             [Rollo]|
|What century did ...|The Normans (Norm...|        10th century|              [10th]|
|Who was the duke ...|The Norman dynast...|William the Conqu...|[William the Conq...|
|Who ruled the duc...|The Norman dynast...|           Richard I|         [Richard I]|
|What religion wer...|The Norman dynast...|            Catholic|          [Catholic]|
|What is the origi...|The English name ...|           

In [ ]:
query.stop()
spark.sql("drop view QA_model")